# Parsing Debug Notebook

This somewhat messy notebook makes it easier to debug the parser, because we can just rerun the cells needed to set up the debugging process with various parameters.

In [1]:
import json
import math
import re
from pathlib import Path

import boto3
import pandas as pd

from parse_990_textract.bucket import open_df
from parse_990_textract.filing import create_roadmap, extract_from_roadmap
from parse_990_textract.models import BoundingBox, TableExtractor
from parse_990_textract.parse import create_extractors, find_item, find_pages
from parse_990_textract.postprocessing import clean_filing, clean_f_i, clean_f_ii, clean_f_iii, postprocess
from parse_990_textract.setup import load_extractor_df
from parse_990_textract.table import extract_table_data, find_table_pages, create_tablemap
from parse_990_textract.utils import get_coordinate, get_regex, cluster_words, columnize, cluster_x

In [2]:
bucket = boto3.resource("s3").Bucket("s3-ocr-990s-demo")

In [3]:
validation_data = pd.read_csv("validation_data.csv", index_col="job_id").fillna("")
validation_data.head()

,pdf_key,other_expenses_c_mgmt_general,payments_affiliates_total,other_expenses_d_prog_service,total_revenue,fees_for_lobbying_services_total,other_employee_benefits_fundraising,total_fundraising_expenses,compensation_officers_prog_service,fees_for_mgmt_services_total,...,travel_total,pension_plan_acc_contrib_prog_service,advertising_promotion_mgmt_general,pension_plan_acc_contrib_mgmt_general,compensation_disq_persons_total,other_salaries_wages_mgmt_general,office_expenses_mgmt_general,fees_for_other_services_fundraising,total_number_other_recipient_foreign_orgs_entities,activities_per_region_totals_total_expenditure
job_id,,,,,,,,,,,,,,,,,,,,,
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4,EIN_760733035_YEAR_2009_FORMTYPE_990.pdf,,,33000.0,1415024,,,4503.0,,,...,18307.0,,,,,,,,,
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc,EIN_363235550_YEAR_2009_FORMTYPE_990.pdf,,,15832.0,9899921,,27161.0,597789.0,295722.0,,...,677722.0,34261.0,30,3462.0,,159156.0,48381.0,22350.0,,
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b,EIN_223134995_YEAR_2010_FORMTYPE_990.pdf,0.0,,10823.0,3370595,,12776.0,263765.0,148689.0,2250.0,...,197092.0,,127381,,396855.0,103312.0,11723.0,23021.0,,
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82,EIN_264320885_YEAR_2009_FORMTYPE_990.pdf,,,,46117,,,669.0,,,...,,,,,,,458.0,,,
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be,EIN_581943161_YEAR_2009_FORMTYPE_990.pdf,581.0,,0.0,583209,,2521.0,14409.0,298484.0,,...,7233.0,,,,,,4010.0,0.0,,


In [4]:
extractor_df = load_extractor_df("990_extractors.csv")
roadmap_df = load_extractor_df("990_roadmap.csv")
schedule_f_tablemap_df = load_extractor_df("schedule_f_table_roadmap.csv")
schedule_f_table_extractor_df = pd.read_csv("schedule_f_table_extractors.csv")
schedule_f_row_extractor_df = pd.read_csv("schedule_f_row_extractors.csv")

In [5]:
PART_I_HEADER = r"\(a\)\s*Region|\(d\)\s*Activities|\(e\)\s*If activity|\(f\)Total expenditures"
PART_II_HEADER = r"\(b\)\s*IRS code|\(c\)\s*Region|\(d\)\s*Purpose|\(f\)\s*Manner|\(h\)\s*Description"
PART_III_HEADER = r"\(b\)\s*Region|\(e\)\s*Manner of cash|\(h\)\s*Method of va"
PART_I_TABLE_NAME = r"Activities per Region"
PART_II_TABLE_NAME = r"Grants to Organizations Outside the United States"
PART_III_TABLE_NAME = r"Grants to Individuals Outside the United States"

In [6]:
filing_rows = []
schedule_f_part_i_rows = []
schedule_f_part_ii_rows = []
schedule_f_part_iii_rows = []


for i, job_id in enumerate(validation_data.index.values):
    print(i)
    print(job_id)
    pdf_key = validation_data.at[job_id, "pdf_key"]
    print(pdf_key)
    
    data = open_df(bucket, job_id)
    lines = data.loc[data["BlockType"] == "LINE"]
    words = data.loc[data["BlockType"] == "WORD"]
    page_map = find_pages(lines)
    roadmap = create_roadmap(
        lines, roadmap_df, page_map
    )
    
    row = extract_from_roadmap(
        words, lines, roadmap, extractor_df, page_map
    )
    row = postprocess(row, job_id, pdf_key, clean_filing)
    filing_rows.append(row)
    
    pages = lines.groupby("Page")
    
    part_i_table = extract_table_data(
        pages, lines, words, PART_I_HEADER, PART_I_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_i_table = postprocess(part_i_table, job_id, pdf_key, clean_f_i)
    if part_i_table is not None:
        schedule_f_part_i_rows.append(
            part_i_table
        )
    part_ii_table = extract_table_data(
        pages, lines, words, PART_II_HEADER, PART_II_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_ii_table = postprocess(part_ii_table, job_id, pdf_key, clean_f_ii)
    if part_ii_table is not None:
        schedule_f_part_ii_rows.append(
            part_ii_table
        )
    part_iii_table = extract_table_data(
        pages, lines, words, PART_III_HEADER, PART_III_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_iii_table = postprocess(part_iii_table, job_id, pdf_key, clean_f_iii)
    if part_iii_table is not None:
        schedule_f_part_iii_rows.append(
            part_iii_table
        )

0
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4
EIN_760733035_YEAR_2009_FORMTYPE_990.pdf


No match for year_formation in L Year of Formation M State of legal domicile
No match for state_of_domicile in M State of legal domicile


1
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc
EIN_363235550_YEAR_2009_FORMTYPE_990.pdf


No match for total_unrelated_biz_revenue in 7a NONE
No match for net_unrelated_biz_taxable_revenue in 7b NONE
No match for benefits_paid_members_expenses in NONE
No match for professional_fundraising_fees_expenses in NONE
No match for grants_foreign_individuals_govt_orgs_total in NONE
No match for benefits_to_members_total in NONE
No match for compensation_disq_persons_total in NONE
No match for fees_for_mgmt_services_total in NONE
No match for fees_for_lobbying_services_total in NONE
No match for fees_for_fundraising_services_total in NONE
No match for fees_for_investment_mgmt_services_total in NONE
No match for royalties_total in NONE
No match for entertainment_travel_govt_officials_total in NONE
No match for interest_total in NONE
No match for payments_affiliates_total in NONE


2
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b
EIN_223134995_YEAR_2010_FORMTYPE_990.pdf
3
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82
EIN_264320885_YEAR_2009_FORMTYPE_990.pdf
4
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be
EIN_581943161_YEAR_2009_FORMTYPE_990.pdf
5
5596f51a999ebbd4cb992f490ceaffcddbac9bce532b911997683ae6897c2797
EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
6
481a62c75776cc7f62075c1d60c230ea01f2dbc5b6224c65cb73db1728e86b00
EIN_954806856_YEAR_2009_FORMTYPE_990.pdf
7
39d270117e4b6354850551c8237421403da7d2b3f5d06657a28f5a9d07febf17
EIN_521238301_YEAR_2010_FORMTYPE_990.pdf


...

No match for activities_per_region_totals_number_of_offices in O


19
aefc7b65c34db330d8d9f56a1226e116b63ee9be7dbdd4ae4c7bea5d87359f97
EIN_362428692_YEAR_2009_FORMTYPE_990.pdf
20
a5a3cbfcf844be8862bbb61ad46d4c795891ab1143e420db5ed99fc79eeb66c9
EIN_271377148_YEAR_2016_FORMTYPE_990.pdf
21
67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebdb32d3186b19d6a6fd5
EIN_42103580_YEAR_2010_FORMTYPE_990.pdf
22
a6529e504df346097da99104a353e977426e018cd5ac33b62cd2dd89c90763c5
EIN_311002913_YEAR_2008_FORMTYPE_990.pdf
23
d1925c2d74adaa3f150ded3ce67dfe7ae3a306f0db0289ad5755a28d801b2b0b
EIN_202408857_YEAR_2010_FORMTYPE_990.pdf


No match for activities_per_region_totals_number_of_offices in Act the


24
01d89ee5d14575c1321b2e4d67431d172ba76212b4a266bdaf474275029fd78b
EIN_521830327_YEAR_2009_FORMTYPE_990.pdf


No match for total_fundraising_expenses in b Total fundraising expenses (Part IX, column (D), line 25)
No match for revenue_less_expenses in <43,775.> End of Year


In [7]:
output_data = pd.concat(filing_rows).reset_index(drop=True).set_index("job_id")

In [8]:
output_data.head()

field_name,index,name,address,city,state,zip,website,gross_receipts,year_formation,state_of_domicile,...,total_functional_expense_fundraising,activities_per_region_totals_number_of_offices,activities_per_region_totals_number_of_employees,activities_per_region_totals_total_expenditure,total_number_recipient_foreign_orgs_listed_as_charities,total_number_other_recipient_foreign_orgs_entities,pdf_key,ein,year,filing_id
job_id,,,,,,,,,,,,,,,,,,,,,
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4,0,MAKE WAY PARTNERS INC,PO BOX 26367,BIRMINGHAM,OX,26367,www MAKEWAYPARTNERS ORG,1426217,,,...,4503,,,,,,EIN_760733035_YEAR_2009_FORMTYPE_990.pdf,760733035,2009,760733035_2009
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc,0,THE FEDERALIST SOCIETY FOR LAW AND,"1015 18TH ST., N.W. 425",WASHINGTON,DC,20036,www. FED-SOC.ORG,11033302,1982,IL,...,597789,,,,,,EIN_363235550_YEAR_2009_FORMTYPE_990.pdf,363235550,2009,363235550_2009
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b,0,Ron Hutchcraft Ministries Inc,PO Box 400,Harnson,AR,20400,www hutchcraft com,3390,1991,NJ,...,263765,,,,,,EIN_223134995_YEAR_2010_FORMTYPE_990.pdf,223134995,2010,223134995_2010
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82,0,FREEDOM 4 24,21430 TIMBERLAKE ROAD STE 101,LYNCHBURG,VA,24502,freedom424 org,61857,2009,VA,...,669,,,,,,EIN_264320885_YEAR_2009_FORMTYPE_990.pdf,264320885,2009,264320885_2009
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be,0,GEORGIA PUBLIC POLICY FOUNDTION,6100 LAKE FORREST DR,LAKE FORREST,GA,30328,www GPPF org,583209,1991,GA,...,14409,,,,,,EIN_581943161_YEAR_2009_FORMTYPE_990.pdf,581943161,2009,581943161_2009


In [9]:
def clean(x):
    x = str(x)
    x = re.sub(r"\.0\b", "", x)
    x = re.sub("\D", "", x)
    return x

In [10]:
def compare_output(to_validate, to_compare, col):
    return pd.DataFrame(
        {
            "extracted": to_validate.loc[col].loc[
                lambda series: series != to_compare.loc[col]
            ],
            "expected": to_compare.loc[col].loc[
                lambda series: series != to_validate.loc[col]
            ],
        }
    )

In [11]:
to_compare = validation_data.set_index("pdf_key").applymap(clean)
to_validate = output_data[validation_data.columns].set_index("pdf_key").applymap(clean)

for col in to_validate.index:
    validated = compare_output(to_validate, to_compare, col)
    if validated.any().any():
        print(col)
        print(f"{validated.shape[0]} mismatched items.")
        print(validated)
        print("-"*79)

EIN_113489123_YEAR_2008_FORMTYPE_990.pdf
2 mismatched items.
              extracted expected
total_revenue      1017  1017506
travel_total        137    13710
-------------------------------------------------------------------------------
EIN_582248383_YEAR_2009_FORMTYPE_990.pdf
1 mismatched items.
                                                   extracted expected
total_number_other_recipient_foreign_orgs_entities        10         
-------------------------------------------------------------------------------


In [12]:
assert False

AssertionError: 

## BREAK

In [13]:
pd.concat(schedule_f_part_i_rows).head(50)[
    [
        "region", "number_offices", "number_employees", 
        "activities_conducted", "specific_type_activity", 
        "total_expenditures", "pdf_key"
    ]
]

field,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures,pdf_key
0,Europe,1,0,theological training,offers a Th M degree,6661,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
1,Totals,1,0,,,6661,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
0,(1),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
1,(2),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
2,(3),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
3,(4),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
4,(5),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
5,(6),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
6,(7),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
7,(8),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf


In [14]:
pd.concat(schedule_f_part_i_rows).tail(50)

field,index,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures,job_id,pdf_key,ein,year,filing_id
25,25,Europe :luding Icelan & Greenland,,,vestments,,,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
26,26,Middle East and North Africa,1,2,Grantmaking,,698609,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
27,27,Middle East and North Africa,,,Progr services,Service centers,26658,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
28,28,Middle East and North Africa,,,Progr services,Academic support,31354,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
29,29,Middle East and North Africa,,,Programservices,Institutional support,7637,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
30,30,Middle East and North Africa,,,Progr Services,Instruction,674753,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
31,31,Middle East and North Africa,,,Progr Services,Research other academicactivity,325683,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
32,32,Middle East and North Africa,,,Progr Services,Student Services,80648,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
33,33,Middle East and North Africa,,,Investment,,,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010
34,34,North merica,,,Fundraising,,33431,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...,EIN_42103580_YEAR_2010_FORMTYPE_990.pdf,42103580,2010,42103580_2010


In [15]:
pd.concat(schedule_f_part_ii_rows).iloc[:50]

field,index,org_name,irs_code,region,grant_purpose,amount_cash,manner_cash,amount_noncash,desc_noncash,method_valuation,job_id,pdf_key,ein,year,filing_id
0,0,(1),,,,,,,,(1),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
1,1,(2),,,,,,,,(2),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
2,2,(3),,,,,,,,(3),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
3,3,(4),,,,,,,,(4),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
4,4,(5),,,,,,,,(5),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
5,5,(6),,,,,,,,(6),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
6,6,(7),,,,,,,,(7),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
7,7,(8),,,,,,,,(8),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
8,8,(9),,,,,,,,(9),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010
9,9,(10),,,,,,,,(10),39d270117e4b6354850551c8237421403da7d2b3f5d066...,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf,521238301,2010,521238301_2010


In [ ]:
pd.concat(schedule_f_part_iii_rows).iloc[:50]

In [16]:
table_test_df = open_df(bucket, "f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2f4a49dd0a81b456649")

In [17]:
test_lines = table_test_df.loc[
    table_test_df["BlockType"] == "LINE"
]
test_words = table_test_df.loc[
    table_test_df["BlockType"] == "WORD"
]
test_pages = test_lines.groupby("Page")

In [18]:
HEADER = PART_II_HEADER
NAME = PART_II_TABLE_NAME

In [19]:
table_pages = find_table_pages(
    test_pages["Text"].agg(lambda words: " ".join(words)),
    HEADER,
)

In [20]:
table_pages

Page
26    26
29    29
Name: Page, dtype: int64

In [52]:
TEST_INDEX = 0
TEST_PAGE = 26

In [53]:
tablemaps = pd.DataFrame(
    {
        "page": table_pages,
        "tablemap": table_pages.map(
            lambda page: create_tablemap(
                test_lines, schedule_f_tablemap_df, page, NAME
            ).dropna()
        ),
    }
)

In [54]:
tablemaps["tablemap"].iloc[TEST_INDEX]

,Top,Left,Top_Default,Left_Default
Item,,,,
(a) Name of organization,0.162970,0.053127,0.05,0.0
(b) IRS code,0.152737,0.178117,0.05,0.0
(c) Region,0.162706,0.296826,0.05,0.0
(d) Purpose,0.153041,0.409870,0.05,0.0
(e) Amount,0.153091,0.521262,0.05,0.0
(f) Manner,0.152864,0.598607,0.05,0.0
(g) Amount,0.150566,0.678248,0.05,0.0
(h) Description,0.150264,0.766059,0.05,0.0
(i) Method,0.149805,0.875547,0.05,0.0


In [56]:
row_extractors = schedule_f_row_extractor_df.loc[
    schedule_f_row_extractor_df["table"] == NAME
]
table_data = schedule_f_table_extractor_df.loc[
    schedule_f_table_extractor_df["table"] == NAME
].iloc[0]

In [57]:
rows = tablemaps.assign(
    extractor=tablemaps["tablemap"].map(
        lambda tablemap: TableExtractor(
            header_top_label=table_data["header_top"],
            top_label=table_data["table_top"],
            bottom_label=table_data["table_bottom"],
            tablemap=tablemap,
            fields=row_extractors["field"],
            field_labels=row_extractors["col_left"]
        )
    )
)

In [58]:
test_extractor = rows["extractor"].iloc[TEST_INDEX]

In [59]:
test_extractor.get_col_spans(test_words, TEST_PAGE)

0                   (0.0, 0.17553073167800903)
1    (0.17553073167800903, 0.2661546468734741)
2    (0.2661546468734741, 0.37508463859558105)
3    (0.37508463859558105, 0.5212618708610535)
4     (0.5212618708610535, 0.5824145376682281)
5     (0.5824145376682281, 0.6782479286193848)
6     (0.6782479286193848, 0.7448297441005707)
7      (0.7448297441005707, 0.855958491563797)
8                     (0.855958491563797, 1.0)
dtype: object

In [60]:
test_extractor.field_labels

6     Top Left Corner
7        (b) IRS code
8          (c) Region
9         (d) Purpose
10         (e) Amount
11         (f) Manner
12         (g) Amount
13    (h) Description
14         (i) Method
Name: col_left, dtype: object

In [61]:
test_extractor.extract_rows(test_words, TEST_PAGE).head(50)

field,org_name,irs_code,region,grant_purpose,amount_cash,manner_cash,amount_noncash,desc_noncash,method_valuation
0,,,CENTRAL AMERICA,,,,,EQUIPMENT AND,
1,,,AND THE CARIBBEAN,PROGRAM SERVICES,"510,426,",WIRE/CHECK,"134,963,",SUPPLIES FMV,
2,,,CENTRAL AMERICA,,,,,EQUIPMENT AND,
3,,,AND THE CARIBBEAN,PROGRAM SERVICES,"1237098,",WIRE/CHECK,"245.225,",SUPPLIES FMV,
4,,,,,,,,EQUIPMENT AND,
5,,,SOUTH ASIA,PROGRAM SERVICES,"1104403,",WIRE/CHECK,"256,456,",SUPPLIES FMV,
6,,,SUB-SAHARAN,,,,,EQUIPMENT AND,
7,,,AFRICA,PROGRAM SERVICES,"950,828,",WIRE/CHECK,4624637.,SUPPLIES FMV,
8,,,SUB-SAHARAN,,,,,EQUIPMENT AND,
9,,,AFRICA,PROGRAM SERVICES,"925,907,",WIRE/CHECK,"223,565,",SUPPLIES FMV,


In [62]:
table_words = test_extractor.get_table_words(test_words, TEST_PAGE)

In [63]:
table_words

,BlockType,Confidence,Geometry,Page,PageClassification,Relationships,Text,TextType,Polygon,Height,Left,Top,Right,Bottom,Midpoint_X,Midpoint_Y,Width,Children,Line_No,File
Id,,,,,,,,,,,,,,,,,,,,
a06ccf24-6f5f-40e1-b438-8d99ae8808e1,WORD,99.504608,{'BoundingBox': {'Height': 0.01968448981642723...,26,None,None,FMV,PRINTED,"[{'X': 0.3420565128326416, 'Y': 0.145734697580...",0.009566,0.854265,0.342057,0.873950,0.351622,0.864108,0.346839,0.019684,None,12,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...
d5766ae0-42fe-4edd-aec1-6938ebf997fa,WORD,99.338219,{'BoundingBox': {'Height': 0.01899544335901737...,26,None,None,FMV,PRINTED,"[{'X': 0.49784764647483826, 'Y': 0.14501953125...",0.009810,0.854980,0.497848,0.873976,0.507657,0.864478,0.502753,0.018995,None,12,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...
10cb444c-5dbe-44f8-bb1a-f1ba1eb4dbca,WORD,99.656029,{'BoundingBox': {'Height': 0.01884985342621803...,26,None,None,FMV,PRINTED,"[{'X': 0.4203152358531952, 'Y': 0.144892573356...",0.009700,0.855107,0.420315,0.873957,0.430015,0.864532,0.425165,0.018850,None,12,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...
ccb6ad23-df0c-49b7-b00c-646628811172,WORD,99.596115,{'BoundingBox': {'Height': 0.01879109628498554...,26,None,None,FMV,PRINTED,"[{'X': 0.6542078256607056, 'Y': 0.144395917654...",0.009522,0.855604,0.654208,0.874395,0.663730,0.865000,0.658969,0.018791,None,12,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...
be1ba02a-29ef-4bcf-9c83-ab3e1fe6ea4f,WORD,99.684349,{'BoundingBox': {'Height': 0.01869604550302028...,26,None,None,FMV,PRINTED,"[{'X': 0.5756934881210327, 'Y': 0.144390463829...",0.009653,0.855610,0.575693,0.874306,0.585347,0.864958,0.580520,0.018696,None,12,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967d97e-e596-497d-98c9-a34c37d2ee03,WORD,99.947380,{'BoundingBox': {'Height': 0.01910469494760036...,26,None,None,THE,PRINTED,"[{'X': 0.26345688104629517, 'Y': 0.70973545312...",0.009739,0.290265,0.263457,0.309369,0.273196,0.299817,0.268326,0.019105,None,77,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...
c9a782df-97e1-4a14-9b7a-f2d7b85b449b,WORD,99.968353,{'BoundingBox': {'Height': 0.01866825297474861...,26,None,None,THE,PRINTED,"[{'X': 0.34212490916252136, 'Y': 0.70939457416...",0.009926,0.290605,0.342125,0.309274,0.352051,0.299940,0.347088,0.018668,None,77,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...
44e88543-7dc7-4d93-9dd3-ae1ee9dab80a,WORD,99.824814,{'BoundingBox': {'Height': 0.03077889047563076...,26,None,None,SOUTH,PRINTED,"[{'X': 0.4204743802547455, 'Y': 0.733448266983...",0.009821,0.266552,0.420474,0.297331,0.430295,0.281941,0.425385,0.030779,None,78,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...


In [65]:
word_clusters = cluster_words(table_words, table_words["Height"].min(), "Midpoint_Y")
[" ".join(word.sort_values(by="Left")["Text"].values) for word in word_clusters]

['CENTRAL AMERICA EQUIPMENT AND',
 'AND THE CARIBBEAN PROGRAM SERVICES 510,426, WIRE/CHECK 134,963, SUPPLIES FMV',
 'CENTRAL AMERICA EQUIPMENT AND',
 'AND THE CARIBBEAN PROGRAM SERVICES 1237098, WIRE/CHECK 245.225, SUPPLIES FMV',
 'EQUIPMENT AND',
 'SOUTH ASIA PROGRAM SERVICES 1104403, WIRE/CHECK 256,456, SUPPLIES FMV',
 'SUB-SAHARAN EQUIPMENT AND',
 'AFRICA PROGRAM SERVICES 950,828, WIRE/CHECK 4624637. SUPPLIES FMV',
 'SUB-SAHARAN EQUIPMENT AND',
 'AFRICA PROGRAM SERVICES 925,907, WIRE/CHECK 223,565, SUPPLIES FMV',
 'SUB-SAHARAN EQUIPMENT AND',
 'AFRICA PROGRAM SERVICES 259,999, WIRE/CHECK 360. SUPPLIES FMV',
 'SUB-SAHARAN EQUIPMENT AND',
 'AFRICA PROGRAM SERVICES 751,580, WIRE/CHECK 688,712, SUPPLIES FMV',
 'SUB-SAHARAN EQUIPMENT AND',
 'AFRICA PROGRAM SERVICES']

In [66]:
def columnize(word_cluster, col_spans):
    return col_spans.map(
        lambda span: word_cluster.loc[
            (word_cluster["Left"].between(*span, inclusive="left"))
        ]
    )


def get_cluster_coords(cluster):
    cluster_coords = {
        "Left": cluster["Left"].min(),
        "Right": cluster["Right"].max(),
        "Height": cluster["Height"].max(),
        "Midpoint_X": cluster["Midpoint_X"].median(),
        "Midpoint_Y": cluster["Midpoint_Y"].median(),
        "Top": cluster["Top"].min(),
        "Bottom": cluster["Bottom"].min(),
    }
    cluster_coords["Width"] = cluster_coords["Right"] - cluster_coords["Left"]
    return cluster_coords


def combine_row(row):
    return pd.Series([
        line.map(
            lambda x: x.sort_values(
                by="Left"
            ).reset_index(drop=True)["Text"].fillna("")
        ).agg(
            lambda x: " ".join(x.values)
        ) + " "
        for line in row
    ]).sum().str.strip()

col_spans = test_extractor.get_col_spans(test_words, TEST_PAGE)

col_spans

0                   (0.0, 0.17553073167800903)
1    (0.17553073167800903, 0.2661546468734741)
2    (0.2661546468734741, 0.37508463859558105)
3    (0.37508463859558105, 0.5212618708610535)
4     (0.5212618708610535, 0.5824145376682281)
5     (0.5824145376682281, 0.6782479286193848)
6     (0.6782479286193848, 0.7448297441005707)
7      (0.7448297441005707, 0.855958491563797)
8                     (0.855958491563797, 1.0)
dtype: object

In [72]:
y_tol = table_words["Height"].max() * 2
x_tol = table_words["Width"].median()
sum_y_delta = y_tol
columnized = columnize(word_clusters[0], col_spans)
last_col_coords = pd.DataFrame.from_records(
    columnized.map(
        get_cluster_coords
    )
)
print(y_tol)
rows = []
current_row = [columnized]
print(" ".join(word_clusters[0].sort_values(by="Left")["Text"].values))
for count, cluster in enumerate(word_clusters[1:]):
    print("-"*50)
    print(" ".join(cluster.sort_values(by="Left")["Text"].values))
    columnized = columnize(cluster, col_spans)
    col_coords = pd.DataFrame.from_records(columnized.map(get_cluster_coords))
    nonempty = col_coords.dropna().index.to_series()
    last_nonempty = last_col_coords.dropna().index.to_series()
    delta_cols = (~nonempty.isin(last_nonempty)).any()
    y_delta = (
        col_coords["Midpoint_Y"].median()
        - last_col_coords["Midpoint_Y"].median()
    )
    sum_y_delta += y_delta
    mean_y_delta = sum_y_delta / (count + 2)
    min_y_delta = min(mean_y_delta, y_tol)
    print("Y Delta:", y_delta)
    print("Min Y Delta:", min_y_delta)
    print("More cols detected", delta_cols and (y_delta > min_y_delta))
    print("Y tolerance exceeded", (y_delta > y_tol) and (y_delta > min_y_delta))
    if (delta_cols or (y_delta > y_tol)) and (y_delta > min_y_delta):
        combined_row = combine_row(current_row)
        print(combined_row)
        rows.append(combined_row)
        current_row = [columnized]
    else:
        current_row.append(columnized)
    last_col_coords = col_coords
print("-"*50)
combined_row = combine_row(current_row)
print(combined_row)
rows.append(combined_row)

0.02573060430586338
CENTRAL AMERICA EQUIPMENT AND
--------------------------------------------------
AND THE CARIBBEAN PROGRAM SERVICES 510,426, WIRE/CHECK 134,963, SUPPLIES FMV
Y Delta: 0.019818151369690895
Min Y Delta: 0.022774377837777138
More cols detected False
Y tolerance exceeded False
--------------------------------------------------
CENTRAL AMERICA EQUIPMENT AND
Y Delta: 0.059323232620954514
Min Y Delta: 0.02573060430586338
More cols detected False
Y tolerance exceeded True
0                                     
1                                     
2    CENTRAL AMERICA AND THE CARIBBEAN
3                     PROGRAM SERVICES
4                             510,426,
5                           WIRE/CHECK
6                             134,963,
7           EQUIPMENT AND SUPPLIES FMV
8                                     
dtype: object
--------------------------------------------------
AND THE CARIBBEAN PROGRAM SERVICES 1237098, WIRE/CHECK 245.225, SUPPLIES FMV
Y Delta: 0.0196406

In [40]:
test_lines.loc[
    test_lines["Text"].str.contains("other\)")
    & test_lines["Page"].between(26, 26)
]

,BlockType,Confidence,Geometry,Page,PageClassification,Relationships,Text,TextType,Polygon,Height,Left,Top,Right,Bottom,Midpoint_X,Midpoint_Y,Width,Children,Line_No,File
Id,,,,,,,,,,,,,,,,,,,,
93fdf60b-42be-4538-b3da-5d23e9b09d28,LINE,98.887878,{'BoundingBox': {'Height': 0.07343416661024094...,26,None,[{'Ids': ['4e954e18-1c48-424f-b864-d1012d0dcc8...,"appraisal, other)",None,"[{'X': 0.1793569028377533, 'Y': 0.132122889161...",0.013385,0.867877,0.179357,0.941311,0.192742,0.904594,0.18605,0.073434,"[4e954e18-1c48-424f-b864-d1012d0dcc80, 9155fc3...",2,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...


In [73]:
test_extractor.get_table_bottom(test_words, TEST_PAGE)

0.8155502043664455

In [75]:
test_words.loc[
    test_words["Text"].str.match("579")
    & test_words["Page"].between(TEST_PAGE, TEST_PAGE)
]

,BlockType,Confidence,Geometry,Page,PageClassification,Relationships,Text,TextType,Polygon,Height,Left,Top,Right,Bottom,Midpoint_X,Midpoint_Y,Width,Children,Line_No,File
Id,,,,,,,,,,,,,,,,,,,,
ca1214ad-a067-4fcd-8898-5de8500322f9,WORD,55.265419,{'BoundingBox': {'Height': 0.04488033056259155...,26,None,None,"579,815,",PRINTED,"[{'X': 0.8099037408828735, 'Y': 0.463886976242...",0.012437,0.536113,0.809904,0.580993,0.82234,0.558553,0.816122,0.04488,None,47,f303b69e79844240beccf4fc5b3cecaa3a4f3024a955b2...


In [47]:
create_tablemap(
    test_lines,
    schedule_f_tablemap_df,
    26,
    NAME
)

,Top,Left,Top_Default,Left_Default
Item,,,,
(a) Name of organization,0.162970,0.053127,0.05,0.0
(b) IRS code,0.152737,0.178117,0.05,0.0
(c) Region,0.162706,0.296826,0.05,0.0
(d) Purpose,0.153041,0.409870,0.05,0.0
(e) Amount,0.153091,0.521262,0.05,0.0
(f) Manner,0.152864,0.598607,0.05,0.0
(g) Amount,0.150566,0.678248,0.05,0.0
(h) Description,0.150264,0.766059,0.05,0.0
(i) Method,0.149805,0.875547,0.05,0.0


In [ ]:
test_extractor.tablemap

In [ ]:
header_words = test_extractor.get_header_words(test_words, 22)

In [ ]:
header_words.sort_values(by="Left")[["Text", "Left", "Right", "Midpoint_Y"]].head(50)

In [ ]:
pd.DataFrame(
    {
        "Gap": header_words.sort_values(by="Left").rolling(2).apply(lambda x: x),
        "Right": header_words.sort_values(by="Left")["Right"],
        "Text": header_words.sort_values(by="Left")["Text"],
    }
)

In [ ]:
def cluster_x(words, tolerance):
    print("Tolerance", tolerance)
    if (tolerance == 0) or (words.shape[0] < 2):
        return [
            [word] for (idx, word)
            in words.sort_values(by="Left").iterrows()
        ]
    groups = []
    sorted_words = words.sort_values(by="Left")
    current_group = [sorted_words.iloc[0]]
    last = sorted_words.iloc[0]["Right"]
    for idx, word in sorted_words.iloc[1:].iterrows():
        if word["Left"] <= (last + tolerance):
            current_group.append(word)
        else:
            print("New Group")
            print("*"*100)
            groups.append(current_group)
            current_group = [word]
        print("Left:", word["Left"])
        print("Right:", word["Right"])
        last = max((last, word["Right"]))
    groups.append(current_group)
    return [pd.DataFrame(group) for group in groups]

In [ ]:
left_right = pd.DataFrame(
    {
        "Left": header_words["Left"].sort_values().reset_index(drop=True),
        "Right": header_words["Right"].sort_values().reset_index(drop=True),
    }
)

header_lines = cluster_x(
    left_right, header_words["Width"].min()*.8
)

In [ ]:
len(header_lines)

In [ ]:
[
    (x["Left"].min(), x["Right"].max())
    for x in header_lines
]

In [ ]:
x_clusters = cluster_x(header_words, header_words["Width"].min()*.8)

In [ ]:
len(x_clusters)

In [ ]:
0.6793086230754852-0.6723970174789429

In [ ]:
[
    words.sort_values(by="Left")["Text"].agg(lambda x: " ".join(x.values))
    for words in x_clusters
]

In [ ]:
left_bounds = pd.Series(
        cluster["Left"].min() for cluster in x_clusters[1:]
)
right_bounds = pd.Series(
    [cluster["Right"].max() for cluster in x_clusters[:-1]]
)
offsets = right_bounds - left_bounds
full_left = pd.concat([pd.Series([0]), left_bounds + offsets]).reset_index(drop=True)
full_right = pd.concat([right_bounds - offsets, pd.Series([1])]).reset_index(drop=True)
col_spans = full_left.combine(full_right, lambda x, y: (x, y))
col_spans

In [ ]:
last_cluster_right = pd.concat([header_words, table_words])["Left"].min()
left_bounds = []
right_bounds = []
for cluster in x_clusters:
    print("Last Cluster Right", last_cluster_right)
    print("Cluster Left", cluster["Left"].min())
    print("Cluster Right", cluster["Right"].max())
    print("*"*50)
    left_bounds.append(last_cluster_right)
    last_cluster_right = max(
        cluster["Right"].max()
        + cluster["Left"].min()
        - last_cluster_right,
        cluster["Right"].max()
    )
    right_bounds.append(last_cluster_right)
col_spans = pd.Series(zip(left_bounds, right_bounds))
col_spans

In [ ]:
header_words["Midpoint_X"].round(2).value_counts()

In [ ]:
def count_crossing_lines(df, left, right):
    return (
        df.loc[
            df["Left"].between(left, right)
            & (df["Right"] > right),
            "Text"
        ].count()
        + df.loc[
            (df["Left"] < left)
            & df["Right"].between(left, right),
            "Text"
        ].count()
    )


def recalculate_intervals(left_interval, right_interval):
    left_left = left_interval[0]
    right_right = right_interval[1]
    total_span = right_right - left_left
    
    

In [ ]:
col_spans.map(
    lambda x: (x[1] + x[0]) / 2
)

In [ ]:
table_words.loc[
    table_words["Left"].between(0.12358373403549194, 0.3558424413204193)
    & (table_words["Midpoint_X"] > 0.3558424413204193),
    "Text"
]

In [ ]:
left_right = pd.DataFrame(
    {
        "left": header_words["Left"].sort_values().reset_index(drop=True),
        "right": header_words["Right"].sort_values().reset_index(drop=True),
    }
)
left_right.rolling(2).apply(lambda x: print(x, "\n" + "*"*20) or 1)

In [ ]:
left_right.head()

In [ ]:
1 and 2

In [ ]:
(test_words.groupby("Page")["Width"].max() / test_words.groupby("Page")["Height"].mean()).index.values

In [ ]:
pd.cut(header_words["Right"], 1000).unique().categories

In [ ]:
sorted_header_words = header_words.sort_values(by="Left")

In [ ]:
len(x_clusters)

In [ ]:
def cluster_x(words, tolerance):
    if (tolerance == 0) or (words.shape[0] < 2):
        return [
            [word] for (idx, word)
            in words.sort_values(by="Midpoint_X").iterrows()
        ]
    groups = []
    sorted_words = words.sort_values(by="Midpoint_X")
    current_group = [sorted_words.iloc[0]]
    last = sorted_words.iloc[0]["Midpoint_X"]
    for idx, word in sorted_words.iloc[1:].iterrows():
        last = (last + word["Midpoint_X"]) / 2
        if word["Midpoint_X"] <= (last + tolerance):
            current_group.append(word)
        else:
            print("New Group")
            print("*"*100)
            groups.append(current_group)
            current_group = [word]
            last = word["Right"]
        print("Word:", word["Text"])
        print("Left:", word["Left"])
        print("Right:", word["Right"])
    groups.append(current_group)
    return [pd.DataFrame(group) for group in groups]

In [ ]:
clusters = cluster_x(header_words, header_words["Width"].mean()*.7)

In [ ]:
len(clusters)

In [ ]:
header_words.sort_values(by="Midpoint_X").rolling(4)["Midpoint_X"].mean()

In [ ]:
left_clusters = cluster_words(header_words, header_words["Width"].mean()*.835, "Left")

In [ ]:
len(left_clusters)

In [ ]:
[
    cluster["Left"].mean() for cluster in left_clusters
]

In [ ]:
def get_col_spans(self, words, page):
    pass

In [ ]:
test_extractor.tablemap

In [ ]:
test_col_spans = pd.Series(
    [
        (0, 0.205227),
        (0.205227, 0.311407),
        (0.311407, 0.420936),
        (0.420936, 0.572975),
        (0.572975, 0.735998),
        (0.735998, 1),
    ]
)

In [ ]:
def find_new_right(df, right):
    return (
        df.loc[
            (df["Right"] > right*1.01)
            & (df["Left"] < right),
            "Left"
        ].min()
    )

In [ ]:
crossing_right = test_col_spans.map(
    lambda x: get_new_right(
        pd.concat([
            test_extractor.get_header_words(test_words, 18),
            test_extractor.get_table_words(test_words, 18),
        ]),
        x[1]
    )
)
crossing_right

In [ ]:
test_col_spans.where(
    crossing_right.isna(),
    test_col_spans.combine(crossing_right, lambda x, y: (x[0], y))
)

In [ ]:
test_col_spans

In [ ]:
init_left = test_extractor.field_labels.map(
    lambda x: get_coordinate(test_extractor.tablemap, x, "Left", "Left_Default")
)
init_right = pd.concat(
    [
        init_left.iloc[1:],
        pd.Series([1]),
    ],
    ignore_index=True,
)

In [ ]:
crossing_right = init_right.map(
    lambda x: find_new_right(
        pd.concat([
            test_extractor.get_header_words(test_words, 25),
            test_extractor.get_table_words(test_words, 25),
        ]),
        x
    )
)
crossing_right

In [ ]:
new_right = init_right.where(
    crossing_right.isna(),
    crossing_right,
)
new_right

In [ ]:
init_left

In [ ]:
init_left.iloc[1:] = new_right.iloc[:-1]

In [ ]:
init_left

In [ ]:
col_spans = init_left.combine(new_right, lambda x, y: (x, y))

In [ ]:
test_lines.loc[
    test_lines["Text"].str.contains("\(a\)\s*Na")
]